## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en analysant la rentabilité brute dans les **principales villes étudiantes françaises** (studios et T1 ≤45m²)

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🔍 Besoins data de Léa
- Évaluer le **taux de vacance locative** en France pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Visualiser les **villes à forte concentration étudiante** en France
- Analyser l'**évolution du prix au m² à l'achat et des loyers étudiants** en France
- Etudier **la rentabilité moyenne en France** en 2024
- Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux)
- Comparer **les quartiers les plus rentables (rentabilité brute)** à ?
- Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?
- Analyser la **localisation des universités/grandes écoles** à ?
- Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? 
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Analyser la **rentabilité locative brute** pour appartements étudiants ≤45m² dans **23 grandes villes françaises**
2. Explorer visuellement les **villes à forte concentration étudiante** et analyser les **taux de vacance locative**
3. Obtenir un **classement des villes** par rentabilité, prix et loyers pour décider où investir avec 200k€

## 📚 Import des bibliothèques ##

In [ ]:
import pandas as pd 
import plotly.express as px
import geopandas as gpd
import requests
import os
import osmnx as ox
from shapely.geometry import Point

# 2e vision : Analyse resserrée sur des villes #

### Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux) ###

In [ ]:
print("Partie Axel")

### Comparer **les quartiers les plus rentables (rentabilité brute)** à ? ###

In [ ]:
print("Partie Lucien")

### Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?  ###

In [ ]:
print("Partie Valentine")

### Analyser la **localisation des universités/grandes écoles** à ? ###

In [ ]:
url_df_enseignement_sup = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits-detail_etablissements.csv"

df_enseignement_sup = pd.read_csv(url_df_enseignement_sup, delimiter=';')

df_enseignement_sup.head()

In [ ]:
df_rennes = df_enseignement_sup[df_enseignement_sup['Commune'] == 'Rennes']

print(f"Nombre de lignes à Rennes :", df_rennes.shape[0])
print(f"Nombres de lignes avec gps à Rennes :", (df_rennes['gps'].isnull() == False).sum())
print(f"Nombres de lignes sans gps à Rennes avant drop:", df_rennes['gps'].isnull().sum())

df_rennes = df_rennes.dropna(subset=['gps'])

print(f"Nombres de lignes sans gps à Rennes après drop:", df_rennes['gps'].isnull().sum())
df_rennes.head()

In [ ]:
# Nettoyage des coordonnées
df_rennes[['lat', 'lon']] = df_rennes['gps'].str.split(',', expand=True)
df_rennes['lat'] = df_rennes['lat'].astype(float)
df_rennes['lon'] = df_rennes['lon'].astype(float)

print(f"Nombre de lignes à Rennes :", df_rennes[['lat', 'lon']].shape[0])
print(f"Nombres de lignes avec lat et lon à Rennes :", ((df_rennes['lat'].isnull() == False) & (df_rennes['lon'].isnull() == False)).sum())
print(f"Nombres de lignes sans lat et lon à Rennes avant drop:", ((df_rennes['lat'].isnull()) & (df_rennes['lon'].isnull())).sum())

# On enlève les lignes sans coordonnées
df_rennes = df_rennes.dropna(subset=['lat', 'lon'])

print(f"Nombres de lignes sans lat et lon à Rennes après drop:", ((df_rennes['lat'].isnull()) & (df_rennes['lon'].isnull())).sum())

In [ ]:
# Transformation en GeoDataFrame 
df_rennes = gpd.GeoDataFrame(
    df_rennes,
    geometry=gpd.points_from_xy(df_rennes.lon, df_rennes.lat),
    crs="EPSG:4326"  # WGS84
)

In [ ]:
df_bordeaux = df_enseignement_sup[df_enseignement_sup['Commune'] == 'Bordeaux']

print(f"Nombre de lignes à Bordeaux :", df_bordeaux.shape[0])
print(f"Nombres de lignes avec gps à Bordeaux :", (df_bordeaux['gps'].isnull() == False).sum())
print(f"Nombres de lignes sans gps à Bordeaux avant drop:", df_bordeaux['gps'].isnull().sum())

df_bordeaux = df_bordeaux.dropna(subset=['gps'])

print(f"Nombres de lignes sans gps à Bordeaux après drop:", df_bordeaux['gps'].isnull().sum())
df_bordeaux.head()

In [ ]:
# Nettoyage des coordonnées
df_bordeaux[['lat', 'lon']] = df_bordeaux['gps'].str.split(',', expand=True)
df_bordeaux['lat'] = df_bordeaux['lat'].astype(float)
df_bordeaux['lon'] = df_bordeaux['lon'].astype(float)

print(f"Nombre de lignes à Bordeaux :", df_bordeaux[['lat', 'lon']].shape[0])
print(f"Nombres de lignes avec lat et lon à Bordeaux :", ((df_bordeaux['lat'].isnull() == False) & (df_bordeaux['lon'].isnull() == False)).sum())
print(f"Nombres de lignes sans lat et lon à Bordeaux avant drop:", ((df_bordeaux['lat'].isnull()) & (df_bordeaux['lon'].isnull())).sum())

# On enlève les lignes sans coordonnées
df_bordeaux = df_bordeaux.dropna(subset=['lat', 'lon'])

print(f"Nombres de lignes sans lat et lon à Bordeaux après drop:", ((df_bordeaux['lat'].isnull()) & (df_bordeaux['lon'].isnull())).sum())

In [ ]:
# Transformation en GeoDataFrame 
df_bordeaux = gpd.GeoDataFrame(
    df_bordeaux,
    geometry=gpd.points_from_xy(df_bordeaux.lon, df_bordeaux.lat),
    crs="EPSG:4326"  # WGS84
)

In [ ]:
url_iris = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/contours-iris-pe.gpkg"
local_path = "contours-iris-pe.gpkg"

# Télécharger une seule fois
if not os.path.exists(local_path):
    print("Téléchargement du fichier...")
    r = requests.get(url_iris)
    with open(local_path, "wb") as f:
        f.write(r.content)

# Lister les couches avec fiona
from fiona import listlayers
layers = listlayers(local_path)
print("Couches disponibles:", layers)

# Charger la couche principale
iris = gpd.read_file(local_path, layer=layers[0])
iris.head()


In [ ]:
url_ref_iris = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/reference_IRIS_geo2025.xlsx"

iris_noms = pd.read_excel(url_ref_iris)

iris_noms = iris_noms.rename(columns={'CODE_IRIS': 'code_iris'})

# fusion avec ton GeoDataFrame (qui contient les codes IRIS)
iris = iris.merge(iris_noms[['code_iris', 'LIB_IRIS', 'LIBCOM']], on='code_iris', how='left')

iris.head()


In [ ]:
# Filtrer
iris_rennes = iris[iris['nom_commune'].str.contains("Rennes", case=False, na=False)].copy()

print(f"Rennes contient", len(iris_rennes), "IRIS")

iris_rennes.head()

In [ ]:
# On met tout dans la même CRS projetée (ex: EPSG:2154 ou utiliser EPSG:4326 par la suite)
iris_rennes = iris_rennes.to_crs(epsg=4326)
df_rennes = df_rennes.to_crs(epsg=4326)


In [ ]:
# spatial join : chaque établissement rattaché à un IRIS
etabs_par_iris_rennes = gpd.sjoin(df_rennes, iris_rennes, how="inner", predicate="within")
etabs_par_iris_rennes.head()

In [ ]:
iris_rennes_stats = iris_rennes.copy()

# Nombre d'établissements
iris_rennes_stats = iris_rennes_stats.merge(
    etabs_par_iris_rennes.groupby('LIB_IRIS').size().reset_index(name='nb_etabs'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etabs':0})
iris_rennes_stats['nb_etabs'] = iris_rennes_stats['nb_etabs'].astype(int)

# Nombre total d'étudiants
iris_rennes_stats = iris_rennes_stats.merge(
    etabs_par_iris_rennes.groupby('LIB_IRIS')['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'].sum().reset_index(name='nb_etudiants'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etudiants':0})
iris_rennes_stats['nb_etudiants'] = iris_rennes_stats['nb_etudiants'].astype(int)

iris_rennes_stats = iris_rennes_stats.to_crs(epsg=2154)  # crs projetée pour calculer surface en m²
iris_rennes_stats['area_m2'] = iris_rennes_stats.geometry.area
iris_rennes_stats['etabs_per_m2'] = iris_rennes_stats['nb_etabs'] / iris_rennes_stats['area_m2']
iris_rennes_stats['students_per_m2'] = iris_rennes_stats['nb_etudiants'] / iris_rennes_stats['area_m2']

In [ ]:
# reconvertir en 4326
iris_plot = iris_rennes_stats.to_crs(epsg=4326)

colorscale = "YlOrRd"  # jaune = faible, rouge = élevé

# Carte initiale avec coloraxis défini
fig = px.choropleth_mapbox(
    iris_plot,
    geojson=iris_plot.__geo_interface__,
    locations=iris_plot.index,
    color='students_per_m2',  # initial color
    hover_name='LIB_IRIS',
    hover_data=['nb_etabs','area_m2','nb_etudiants'],
    mapbox_style="carto-positron",
    center={"lat":48.117, "lon":-1.677},
    zoom=12,
    opacity=0.6,
)

# On associe explicitement la trace à coloraxis et fixe les limites
fig.update_traces(
    coloraxis="coloraxis"
)

# Définition globale de coloraxis
fig.update_layout(
    coloraxis=dict(
        colorscale=colorscale,
        cmin=iris_plot['students_per_m2'].min(),
        cmax=iris_plot['students_per_m2'].max(),
        colorbar=dict(title="Étudiants/m²")
    )
)

# Points établissements
fig.add_scattermapbox(
    lat=df_rennes['lat'],
    lon=df_rennes['lon'],
    mode='markers',
    marker=dict(size=6, color='blue'),
    text=etabs_par_iris_rennes["libellé de l'établissement"],
    name='Établissements OSM'
)

# Boutons interactifs : on ne touche qu'à z, zmin, zmax
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            x=0.0, y=1.05, showactive=True,
            buttons=[
                dict(
                    label="Densité",
                    method="update",
                    args=[
                        {"z": [iris_plot['students_per_m2']]},
                        {"coloraxis.cmin": iris_plot['students_per_m2'].min(),
                         "coloraxis.cmax": iris_plot['students_per_m2'].max(),
                         "coloraxis.colorbar.title": "Étudiants/m²"}
                    ]
                ),
                dict(
                    label="Nombre d'étudiants",
                    method="update",
                    args=[  
                        {"z": [iris_plot['nb_etudiants']]},
                        {"coloraxis.cmin": iris_plot['nb_etudiants'].min(),
                         "coloraxis.cmax": iris_plot['nb_etudiants'].max(),
                         "coloraxis.colorbar.title": "Nombre d'étudiants"}
                    ]
                ),
            ]
        )
    ]
)

fig.update_layout(margin={"r":0,"t":75,"l":0,"b":0}, title="Étudiants sup. par IRIS - Rennes")
fig.show()


In [ ]:
colorscale = "YlOrRd"  # jaune = faible, rouge = élevé

# Top 10 IRIS par densité (par défaut)
top10_density = iris_rennes_stats.sort_values(by='students_per_m2', ascending=False).head(10)

# Top 10 IRIS par nombre d'étudiants
top10_students = iris_rennes_stats.sort_values(by='nb_etudiants', ascending=False).head(10)

# Carte initiale avec coloraxis défini
fig = px.bar(
    top10_density,
    x='LIB_IRIS',
    y='students_per_m2',
    color='students_per_m2',
    hover_data=['nb_etudiants','area_m2','nb_etabs'],
    color_continuous_scale=colorscale,
    title="Top 10 IRIS les plus denses en étudiants - Rennes"
)

# On associe explicitement la trace à coloraxis et fixe les limites
fig.update_traces(marker_coloraxis="coloraxis")

# Définition globale de coloraxis
fig.update_layout(
    coloraxis=dict(
        colorscale=colorscale,
        reversescale=False,  # rouge = élevé, jaune = faible
        cmin=top10_density['students_per_m2'].min(),
        cmax=top10_density['students_per_m2'].max(),
        colorbar=dict(title="Étudiants/m²")
    )
)

# Boutons interactifs : on ne touche qu'à x, y, marker.color et coloraxis
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            x=0.0, y=1.05, showactive=True,
            buttons=[
                dict(
                    label="Densité",
                    method="update",
                    args=[
                        {"x": [top10_density['LIB_IRIS']],
                         "y": [top10_density['students_per_m2']],
                         "marker.color": [top10_density['students_per_m2']]},
                        {"coloraxis.cmin": top10_density['students_per_m2'].min(),
                         "coloraxis.cmax": top10_density['students_per_m2'].max(),
                         "coloraxis.colorbar.title": "Étudiants/m²",
                         "yaxis.title": "Étudiants / m²"}
                    ]
                ),
                dict(
                    label="Nombre d'étudiants",
                    method="update",
                    args=[
                        {"x": [top10_students['LIB_IRIS']],
                         "y": [top10_students['nb_etudiants']],
                         "marker.color": [top10_students['nb_etudiants']]},
                        {"coloraxis.cmin": top10_students['nb_etudiants'].min(),
                         "coloraxis.cmax": top10_students['nb_etudiants'].max(),
                         "coloraxis.colorbar.title": "Nombre d'étudiants",
                         "yaxis.title": "Nombre d'étudiants"}
                    ]
                ),
            ]
        )
    ]
)

fig.update_layout(xaxis_title="Quartier (IRIS)", yaxis_title="Étudiants / m²")
fig.show()


In [ ]:
# Filtrer
iris_bordeaux = iris[iris['nom_commune'].str.contains("Bordeaux", case=False, na=False)].copy()

print(f"Bordeaux contient", len(iris_bordeaux), "IRIS")

iris_bordeaux.head()

In [ ]:
# On met tout dans la même CRS projetée (ex: EPSG:2154 ou utiliser EPSG:4326 par la suite)
iris_bordeaux = iris_bordeaux.to_crs(epsg=4326)
df_bordeaux = df_bordeaux.to_crs(epsg=4326)


In [ ]:
# spatial join : chaque établissement rattaché à un IRIS
etabs_par_iris_bordeaux = gpd.sjoin(df_bordeaux, iris_bordeaux, how="inner", predicate="within")
etabs_par_iris_bordeaux.head()

In [ ]:
iris_bordeaux_stats = iris_bordeaux.copy()

# Nombre d'établissements
iris_bordeaux_stats = iris_bordeaux_stats.merge(
    etabs_par_iris_bordeaux.groupby('LIB_IRIS').size().reset_index(name='nb_etabs'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etabs':0})
iris_bordeaux_stats['nb_etabs'] = iris_bordeaux_stats['nb_etabs'].astype(int)

# Nombre total d'étudiants
iris_bordeaux_stats = iris_bordeaux_stats.merge(
    etabs_par_iris_bordeaux.groupby('LIB_IRIS')['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'].sum().reset_index(name='nb_etudiants'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etudiants':0})
iris_bordeaux_stats['nb_etudiants'] = iris_bordeaux_stats['nb_etudiants'].astype(int)

iris_bordeaux_stats = iris_bordeaux_stats.to_crs(epsg=2154)  # crs projetée pour calculer surface en m²
iris_bordeaux_stats['area_m2'] = iris_bordeaux_stats.geometry.area
iris_bordeaux_stats['etabs_per_m2'] = iris_bordeaux_stats['nb_etabs'] / iris_bordeaux_stats['area_m2']
iris_bordeaux_stats['students_per_m2'] = iris_bordeaux_stats['nb_etudiants'] / iris_bordeaux_stats['area_m2']



In [ ]:
# reconvertir en 4326
iris_plot = iris_bordeaux_stats.to_crs(epsg=4326)

colorscale = "YlOrRd"  # jaune = faible, rouge = élevé

# Carte initiale avec coloraxis défini
fig = px.choropleth_mapbox(
    iris_plot,
    geojson=iris_plot.__geo_interface__,
    locations=iris_plot.index,
    color='students_per_m2',  # initial color
    hover_name='LIB_IRIS',
    hover_data=['nb_etabs','area_m2','nb_etudiants'],
    mapbox_style="carto-positron",
    center={"lat":44.8378, "lon":-0.5792},
    zoom=12,
    opacity=0.6,
)

# On associe explicitement la trace à coloraxis et fixe les limites
fig.update_traces(
    coloraxis="coloraxis"
)

# Définition globale de coloraxis
fig.update_layout(
    coloraxis=dict(
        colorscale=colorscale,
        cmin=iris_plot['students_per_m2'].min(),
        cmax=iris_plot['students_per_m2'].max(),
        colorbar=dict(title="Étudiants/m²")
    )
)

# Points établissements
fig.add_scattermapbox(
    lat=df_bordeaux['lat'],
    lon=df_bordeaux['lon'],
    mode='markers',
    marker=dict(size=6, color='blue'),
    text=etabs_par_iris_bordeaux["libellé de l'établissement"],
    name='Établissements OSM'
)

# Boutons interactifs : on ne touche qu'à z, zmin, zmax
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            x=0.0, y=1.05, showactive=True,
            buttons=[
                dict(
                    label="Densité",
                    method="update",
                    args=[
                        {"z": [iris_plot['students_per_m2']]},
                        {"coloraxis.cmin": iris_plot['students_per_m2'].min(),
                         "coloraxis.cmax": iris_plot['students_per_m2'].max(),
                         "coloraxis.colorbar.title": "Étudiants/m²"}
                    ]
                ),
                dict(
                    label="Nombre d'étudiants",
                    method="update",
                    args=[
                        {"z": [iris_plot['nb_etudiants']]},
                        {"coloraxis.cmin": iris_plot['nb_etudiants'].min(),
                         "coloraxis.cmax": iris_plot['nb_etudiants'].max(),
                         "coloraxis.colorbar.title": "Nombre d'étudiants"}
                    ]
                ),
            ]
        )
    ]
)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0}, title="Étudiants sup. par IRIS - Bordeaux")
fig.show()


In [ ]:
colorscale = "YlOrRd"  # jaune = faible, rouge = élevé

# Top 10 IRIS par densité (par défaut)
top10_density_bordeaux = iris_bordeaux_stats.sort_values(by='students_per_m2', ascending=False).head(10)

# Top 10 IRIS par nombre d'étudiants
top10_students_bordeaux = iris_bordeaux_stats.sort_values(by='nb_etudiants', ascending=False).head(10)

# Carte initiale avec coloraxis défini
fig = px.bar(
    top10_density_bordeaux,
    x='LIB_IRIS',
    y='students_per_m2',
    color='students_per_m2',
    hover_data=['nb_etudiants','area_m2','nb_etabs'],
    color_continuous_scale=colorscale,
    title="Top 10 IRIS les plus denses en étudiants - Rennes"
)

# On associe explicitement la trace à coloraxis et fixe les limites
fig.update_traces(marker_coloraxis="coloraxis")

# Définition globale de coloraxis
fig.update_layout(
    coloraxis=dict(
        colorscale=colorscale,
        reversescale=False,  # rouge = élevé, jaune = faible
        cmin=top10_density['students_per_m2'].min(),
        cmax=top10_density['students_per_m2'].max(),
        colorbar=dict(title="Étudiants/m²")
    )
)

# Boutons interactifs : on ne touche qu'à x, y, marker.color et coloraxis
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            x=0.0, y=1.05, showactive=True,
            buttons=[
                dict(
                    label="Densité",
                    method="update",
                    args=[
                        {"x": [top10_density_bordeaux['LIB_IRIS']],
                         "y": [top10_density_bordeaux['students_per_m2']],
                         "marker.color": [top10_density_bordeaux['students_per_m2']]},
                        {"coloraxis.cmin": top10_density_bordeaux['students_per_m2'].min(),
                         "coloraxis.cmax": top10_density_bordeaux['students_per_m2'].max(),
                         "coloraxis.colorbar.title": "Étudiants/m²",
                         "yaxis.title": "Étudiants / m²"}
                    ]
                ),
                dict(
                    label="Nombre d'étudiants",
                    method="update",
                    args=[
                        {"x": [top10_students_bordeaux['LIB_IRIS']],
                         "y": [top10_students_bordeaux['nb_etudiants']],
                         "marker.color": [top10_students_bordeaux['nb_etudiants']]},
                        {"coloraxis.cmin": top10_students_bordeaux['nb_etudiants'].min(),
                         "coloraxis.cmax": top10_students_bordeaux['nb_etudiants'].max(),
                         "coloraxis.colorbar.title": "Nombre d'étudiants",
                         "yaxis.title": "Nombre d'étudiants"}
                    ]
                ),
            ]
        )
    ]
)

fig.update_layout(xaxis_title="Quartier (IRIS)", yaxis_title="Étudiants / m²")
fig.show()


### Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? ###

In [ ]:
print("Partie Adam et Valentine")